This short document serves as an introduction to DynamicalSystems.jl. We'll need the following packages:

In [ ]:
using Pkg; Pkg.add(["DynamicalSystems", "OrdinaryDiffEq"]);

*for more details beyond this short document, see the [documentation of DynamicalSystems.jl](https://juliadynamics.github.io/DynamicalSystems.jl/dev/#Getting-started)*


# Core structures of DynamicalSystems.jl

There are two core structures: `DynamicalSystem`, which represents a dynamical system, and `Dataset` which represents a collection of points in the state space.

Here we'll create two dynamical systems:

1. The Henon map, which is discrete and low-dimensional
2. The Lorenz96 model, which is continuous and high-dimensional

$$
\text{Henon map:}\quad
\begin{array}{l}
x_{n+1} = 1 - ax^2_n+y_n \\
y_{n+1} = bx_n
\end{array}
\quad\quad
\text{Lorenz96:}\quad
\frac{dx_i}{dt} \equiv \dot{x}_i = (x_{i+1}-x_{i-2})x_{i-1} - x_i + F, \,\, i\in \{1,\ldots,N\}
$$
*(notice that $n$ counts the discrete time, while $i$ counts the different variables of the Lorenz96 model which is in continuous time)*

For this tutorial we'll use parameters $(a,b) = (1.4, 0.3)$ for the Henon map, and $(F, N) = (8, 5)$ for the Lorenz96 model. 

Let's start with the discrete one. How to create a dynamical system is discussed in detail in the [documentation string of `DynamicalSystem`](https://juliadynamics.github.io/DynamicalSystems.jl/dev/ds/general/#DynamicalSystemsBase.DynamicalSystem).

In [69]:
using DynamicalSystems

In [70]:
function henon_rule(u, p, n) # here `n` is "time", but we don't use it.
    x, y = u # system state
    a, b = p # system parameters
    xn = 1.0 - a*x^2 + y
    yn = b*x
    return SVector(xn, yn)
end

henon_rule (generic function with 1 method)

In [71]:
p = [1.4, 0.3]
u0 = [0.2, 0.3]

2-element Vector{Float64}:
 0.2
 0.3

In [72]:
henon = DiscreteDynamicalSystem(henon_rule, u0, p)

2-dimensional discrete dynamical system
 state:       [0.2, 0.3]
 rule f:      henon_rule
 in-place?    false
 jacobian:    ForwardDiff
 parameters:  [1.4, 0.3]

This `henon`, which is an instance of a `DynamicalSystem`, contains all information necessary and can be passed to many of the 100s of functions of the library. The best place to start is the function `trajectory` function, which evolves the dynamical system in time and returns the trajectory as a `Dataset`.

In [73]:
A = trajectory(henon, 100) # evolve for `100` steps

2-dimensional Dataset{Float64} with 101 points
  0.2         0.3
  1.244       0.06
 -1.10655     0.3732
 -0.341035   -0.331965
  0.505208   -0.102311
  0.540361    0.151562
  0.742777    0.162108
  0.389703    0.222833
  1.01022     0.116911
 -0.311842    0.303065
  1.16692    -0.0935526
 -0.99994     0.350076
 -0.0497562  -0.299982
  ⋮          
 -0.359887    0.307745
  1.12642    -0.107966
 -0.884314    0.337926
  0.243109   -0.265294
  0.651963    0.0729327
  0.477855    0.195589
  0.875905    0.143356
  0.0692622   0.262772
  1.25606     0.0207787
 -1.18797     0.376817
 -0.598954   -0.35639
  0.141365   -0.179686

The type of the output of `trajectory` is a `Dataset`, which is the 2nd core structure of DynamicalSystems.jl. It is printed like a matrix where each column is the timeseries of each dynamic variable. In reality, it is a vector of statically sized vectors (for performance reasons). When iterated or indexed with 1 index, it behaves like a vector of vectors. When indexed with two indices, it behaves like a matrix.

In [74]:
A[2]

2-element SVector{2, Float64} with indices SOneTo(2):
 1.244
 0.06

In [75]:
A[2:5]

2-dimensional Dataset{Float64} with 4 points
  1.244      0.06
 -1.10655    0.3732
 -0.341035  -0.331965
  0.505208  -0.102311

In [76]:
A[2:5, 2]

4-element Vector{Float64}:
  0.06
  0.3732
 -0.3319651199999999
 -0.10231059085086688

In [77]:
for (i, point) in enumerate(A)
    @show point
    i > 5 && break
end

point = [0.2, 0.3]
point = [1.244, 0.06]
point = [-1.1065503999999997, 0.3732]
point = [-0.34103530283622296, -0.3319651199999999]
point = [0.5052077711071681, -0.10231059085086688]
point = [0.5403605603672313, 0.1515623313321504]


In [78]:
x, y = columns(A)

([0.2, 1.244, -1.1065503999999997, -0.34103530283622296, 0.5052077711071681, 0.5403605603672313, 0.7427769820516068, 0.3897034650181992, 1.0102167877094148, -0.31184210193244044  …  -0.8843143777780145, 0.2431090843448369, 0.6519628490192166, 0.47785494620169566, 0.8759053652525692, 0.06926219143097734, 1.256055457949221, -1.1879667813923445, -0.5989544657835943, 0.14136493266734151], [0.3, 0.06, 0.3732, -0.3319651199999999, -0.10231059085086688, 0.1515623313321504, 0.1621081681101694, 0.22283309461548204, 0.11691103950545975, 0.30306503631282444  …  0.33792577058772055, -0.2652943133334043, 0.07293272530345107, 0.19558885470576498, 0.1433564838605087, 0.2627716095757708, 0.0207786574292932, 0.3768166373847663, -0.35639003441770334, -0.1796863397350783])

Okay, let's also define the Lorenz96 system, now in in-place form

In [80]:
function lorenz96_rule(du, u, p, t)
    F = p[1]; N = length(u)
    # 3 edge cases
    du[1] = (u[2] - u[N - 1]) * u[N] - u[1] + F
    du[2] = (u[3] - u[N]) * u[1] - u[2] + F
    du[N] = (u[1] - u[N - 2]) * u[N - 1] - u[N] + F
    # then the general case
    for n in 3:(N - 1)
        du[n] = (u[n + 1] - u[n - 2]) * u[n - 1] - u[n] + F
    end
    return nothing # always `return nothing` for in-place form!
end

lorenz96_rule (generic function with 1 method)

In [79]:
u0 = rand(5)
p = [8.0]
lorenz96 = ContinuousDynamicalSystem(lorenz96_rule, u0, p)

5-dimensional continuous dynamical system
 state:       [0.460626, 0.395062, 0.13681, 0.0301405, 0.40919]
 rule f:      lorenz96_rule
 in-place?    true
 jacobian:    ForwardDiff
 parameters:  [8.0]

In [81]:
tr = trajectory(lorenz96, 100.0)

5-dimensional Dataset{Float64} with 10001 points
  0.460626   0.395062  0.13681   0.0301405   0.40919
  0.537256   0.469382  0.213205  0.109468    0.484945
  0.61337    0.542758  0.288537  0.188021    0.560201
  0.688961   0.615197  0.362823  0.265812    0.634963
  0.764026   0.686706  0.436079  0.342852    0.709238
  0.838557   0.75729   0.508321  0.419156    0.78303
  0.912548   0.826954  0.579564  0.494736    0.856349
  0.985994   0.8957    0.649824  0.569607    0.929202
  1.05889    0.963531  0.719113  0.643784    1.0016
  1.13122    1.03045   0.787447  0.717284    1.07355
  1.20298    1.09645   0.854839  0.790123    1.14506
  1.27417    1.16154   0.921302  0.862319    1.21615
  1.34477    1.2257    0.98685   0.93389     1.28683
  ⋮                                         
 -0.554116  -0.961738  2.97148   8.17844     2.5477
 -0.690047  -0.87586   2.94063   8.27512     2.30888
 -0.802619  -0.793092  2.91573   8.36035     2.06123
 -0.891157  -0.713824  2.89704   8.43447     1.80654
 

Continuous dynamical systems are evolved through DifferentialEquations.jl. This means that for any function in DynamicalSystems.jl that takes as an input a `ContinuousDynamicalSystem`, you can tune the solver properties to your heart's content using any of the [ODE solvers](https://diffeq.sciml.ai/latest/solvers/ode_solve/) and any of the [common solver options](https://diffeq.sciml.ai/latest/basics/common_solver_opts/). For example:

In [82]:
using OrdinaryDiffEq # accessing the ODE solvers

In [84]:
diffeq = (alg = Vern9(), abstol = 1e-9, reltol = 1e-9)

(alg = Vern9(true), abstol = 1.0e-9, reltol = 1.0e-9)

In [85]:
tr = trajectory(lorenz96, 100.0; diffeq...) # expand keywords with `...`

5-dimensional Dataset{Float64} with 10001 points
 0.460626   0.395062   0.13681    0.0301405  0.40919
 0.537256   0.469382   0.213204   0.109468   0.484945
 0.61337    0.542758   0.288536   0.188021   0.560202
 0.688962   0.615197   0.362822   0.265812   0.634964
 0.764026   0.686706   0.436078   0.342853   0.709238
 0.838557   0.75729    0.508321   0.419156   0.78303
 0.912548   0.826954   0.579565   0.494736   0.856349
 0.985993   0.8957     0.649824   0.569607   0.929201
 1.05889    0.963531   0.719114   0.643784   1.0016
 1.13122    1.03045    0.787447   0.717284   1.07355
 1.20298    1.09645    0.854839   0.790123   1.14506
 1.27417    1.16154    0.921302   0.862319   1.21615
 1.34477    1.2257     0.98685    0.933889   1.28683
 ⋮                                           
 8.29758    1.84508   -0.541735  -1.12559    2.76683
 8.37165    1.6304    -0.619604  -1.04063    2.72283
 8.43578    1.41335   -0.676236  -0.958325   2.68533
 8.49048    1.19549   -0.711667  -0.878644   2.65453

# Using the rest of the library
Once you have a basic understanding of these two structures `Dataset` and `DynamicalSystem`, using the rest of the library is straightforward. Just have a look at the documentation string of the function you want to use, and you are good to go. 

## Example: using a `DynamicalSystem`
Some of the things you could do with a `DynamicalSystem` instance is (for example) calculate the Lyapunov exponents of an initial condition. For this, you'd first read the [documentation string of the `lyapunovspectrum` function](https://juliadynamics.github.io/DynamicalSystems.jl/dev/chaos/lyapunovs/#ChaosTools.lyapunovspectrum).

In [86]:
lyapunovspectrum(lorenz96, 10000)

5-element Vector{Float64}:
  0.47013665574330804
  0.00023264605674148828
 -0.5148209992440717
 -1.2975145256968428
 -3.658034079043204

The documentation string of `lyapunovspectrum` highlights why DynamicalSystems.jl is a library, not just a software. 

Another example could be to use `poincaresos(ds, ...)` to get a Poincare section of the dynamical system, or to obtain the fixed points and their stability:

In [88]:
box = IntervalBox(-2..2, -2..2)
fp, eigs, stable = fixedpoints(henon, box)
for (i, p) in enumerate(fp)
    println("Fixed point $(p) has stability $(stable[i])")
end

Fixed point [0.6313544770895048, 0.1894063431268514] has stability false
Fixed point [-1.1313544770895048, -0.33940634312685136] has stability false


## Example: using a `Dataset`
Similarly, once you have a `Dataset`, there are many things you could be doing with it. Notice that a `Dataset` is not only derived from the evolution of a dynamical system using `trajectory`. You could get a `Dataset` by loading in some experimentally measured data, or generate one via delay coordinates embedding of an existing timeseries. E.g.,

In [89]:
x = tr[:, 1] # first variable timeseries
τ = estimate_delay(x, "mi_min")
E = embed(x, 5, τ)

5-dimensional Dataset{Float64} with 9885 points
 0.460626   2.4424   3.51491   3.44466   7.96079
 0.537256   2.49978  3.52171   3.48209   8.22117
 0.61337    2.55619  3.52613   3.52621   8.4743
 0.688962   2.61158  3.5282    3.57726   8.71752
 0.764026   2.6659   3.52796   3.63543   8.94822
 0.838557   2.71913  3.52551   3.70088   9.16379
 0.912548   2.7712   3.52093   3.77375   9.36155
 0.985993   2.82206  3.51434   3.85418   9.53873
 1.05889    2.87168  3.50589   3.9423    9.69245
 1.13122    2.91998  3.49576   4.03828   9.81969
 1.20298    2.96693  3.48415   4.1423    9.91742
 1.27417    3.01244  3.47128   4.25463   9.98277
 1.34477    3.05647  3.45742   4.37557  10.0132
 ⋮                                      
 5.31655   -4.44653  0.617003  1.70569   8.29758
 5.18398   -4.55916  0.715355  1.97012   8.37165
 5.01617   -4.64546  0.780624  2.24537   8.43578
 4.81238   -4.70575  0.814272  2.52949   8.49048
 4.57243   -4.73995  0.818524  2.82056   8.53624
 4.29667   -4.74759  0.796294  

From a `Dataset`, you can then e.g., calculate its entropy based on a given definition of probabilities

In [90]:
est = NaiveKernel(0.2)
g = genentropy(E, est)

6.725689594594948

or numerically estimate its fractal dimension

In [91]:
grassberger_dim(E) # automatically estimates suitable radii (NOT to be trusted!)

2.235155380156463

or create a recurrence matrix out of it

In [92]:
R = RecurrenceMatrix(E, 0.5)

       RecurrenceMatrix of size (9885, 9885) with 39345 entries    
    ┌────────────────────────────────────────────────────────────┐ 
    │  '    .          .          :..:     :      '..       .:.::│
    │       .                    '::.              '      ' ::::.│
    │                    ..                             :.::' .  │
    │     '.            '::                            .:''.     │
    │                  .        : ''..':':'     :' '..:'         │
    │       '                    :::'             .:''.       ''.│
    │          .                .:'  . ...:   . .:'   .         '│
    │        .               .:'       .'    .:::                │
    │       ''    '        .''      .      .:'                 . │
    │                          .. .' .....:'    .:    :'        '│
    │                          '.    .::':    .':'    :          │
    │       .          .        '   .:'''' .    '   .:'       .. │
    │  .                         ::::     :       .:: :     

and then get some recurrence quantifiers

In [93]:
rqa(R)

RQA parameters in Dict{Symbol, Float64} with 14 entries:
  :DIV   => 0.00168067
  :LAM   => 0.453281
  :Vmax  => 8.0
  :VENTR => 0.893277
  :Lmax  => 595.0
  :MRT   => 935.783
  :NMPRT => 6704.0
  :RR    => 0.000301525
  :RTE   => 2.74542
  :TT    => 2.45091
  :L     => 19.8419
  :ENTR  => 3.3352
  :DET   => 0.996708
  :TREND => -7.31104e-5

From here, the sky is the limit. The best way forwards is to simply visit the [Contents page](https://juliadynamics.github.io/DynamicalSystems.jl/dev/contents/), which lists everything possible with DynamicalSystems.jl.